In [5]:
import pymongo as pm
import pandas as pd
import psycopg2

In [6]:
mydb=psycopg2.connect(host='localhost',
                   user='postgres',
                   password='Adhi@11004',
                   database='airbnb',
                   port='5432')

In [7]:
Client = client=pm.MongoClient("mongodb://localhost:27017/")
db=client['sample_airbnb']
collection = db['listingsAndReviews']

### Total documents

In [8]:
Documents = [i for i in collection.find()]
len(Documents)

5555

### Features
Id: Unique identifier for each listing.

Name: The title or name of the listing.

Host Id: Unique identifier for the host of the listing.

Host Name: The name of the host.

Neighbourhood: The specific neighborhood where the listing is located.

Latitude: The latitude coordinate of the listing's location.

Longitude: The longitude coordinate of the listing's location.

Room Type: The type of room being listed (e.g., "Entire home/apartment", "Private room", "Shared room").

Price: The price to rent the listing.

Minimum Nights: The minimum number of nights a guest must book for the listing.

Maximum Nights: The Maximun number of nights a guest must book for the listing.

Number Of Reviews: The total number of reviews the listing has received.

Availability 30: The number of days the listing is available for booking within the next 30 days

Availability 60: The number of days the listing is available for booking within the next 60 days

Availability 90: The number of days the listing is available for booking within the next 90 days

Availability 365: The number of days the listing is available for booking within the next 365 days

Rating : ratings given by guests to the listings.

# Collecting relevent data

In [9]:
res = [i for i in collection.find({},{'_id':1,'name':1,'description':1,'host.host_id':1,'host.host_name':1,'host.host_neighbourhood':1,'address.location.coordinates':1,'price':1, 'availability.availability_30':1 , 'availability.availability_60':1 ,'availability.availability_90':1 ,'availability.availability_365':1,'room_type':1,'minimum_nights':1,'maximum_nights':1 ,'number_of_reviews':1,'host.host_total_listings_count':1,'review_scores.review_scores_rating':1,'review':1,'amenities':1,'property_type':1})]
     

In [10]:
data = {'Id':[],'Name':[],'Description':[],'Property Type':[],'Room Type':[],'Minimum Nights':[],'Maximum Nights':[],'Number Of Reviews':[],'Amenities':[],'Price':[],'Host ID':[],'Host Name':[],'Host Neighbourhood':[],'Host Total Listings Count':[],'Longitide':[],'Latitude':[],'Availability 30':[],'Availability 60':[],'Availability 90':[],'Availability 365':[],'Rating':[]}
     

In [11]:
data

{'Id': [],
 'Name': [],
 'Description': [],
 'Property Type': [],
 'Room Type': [],
 'Minimum Nights': [],
 'Maximum Nights': [],
 'Number Of Reviews': [],
 'Amenities': [],
 'Price': [],
 'Host ID': [],
 'Host Name': [],
 'Host Neighbourhood': [],
 'Host Total Listings Count': [],
 'Longitide': [],
 'Latitude': [],
 'Availability 30': [],
 'Availability 60': [],
 'Availability 90': [],
 'Availability 365': [],
 'Rating': []}

In [12]:
res[0]

{'_id': '1003530',
 'name': 'New York City - Upper West Side Apt',
 'description': "Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement. New York City! Great neighborhood - many terrific restaurants, bakeries, bagelries. Within easy walking distance are restaurants with the cuisines from India, Thailand, Japan, China, Mexico, South America and Europe.  As well as the many small independent stores that line Broadway, there chain stores such as Urban Outfitters (clothing), Whole Foods (groceries), Sephora (cosmetics), Michaels (crafts), and Modell's (sporting goods). Equidistant to Central Park and Riverside Park which have walking/running/biking trails as well as tennis and racquet ball courts. 10-15 blocks from Columbia University between Broadway and Amsterdam. The International Hostel across the street is an airport shuttle bus and double-decker tour bus stop.  The Hostel also has 

# Preprocessing

In [13]:
for i in res:

  # Id
  data['Id'].append(i['_id'])

  # Name
  data['Name'].append(i['name'])

  # Description
  data['Description'].append(i['description'])

  # Property Type
  data['Property Type'].append(i['property_type'])

  # Room Type
  data['Room Type'].append(i['room_type'])

  # Minimun Nights
  data['Minimum Nights'].append(i['minimum_nights'])

  # Maximun Nights
  data['Maximum Nights'].append(i['maximum_nights'])

  # Number Reviews
  data['Number Of Reviews'].append(i['number_of_reviews'])

  # Amenities
  data['Amenities'].append(i['amenities'])

  # Price
  data['Price'].append(i['price'])

  # Host details
  data['Host ID'].append(i['host']['host_id'])
  data['Host Name'].append(i['host']['host_name'])
  data['Host Neighbourhood'].append(i['host']['host_neighbourhood'])
  data['Host Total Listings Count'].append(i['host']['host_total_listings_count'])

  # Longitude and Latitude
  data['Longitide'].append(i['address']['location']['coordinates'][0])
  data['Latitude'].append(i['address']['location']['coordinates'][1])

  # Availability
  data['Availability 30'].append(i['availability']['availability_30'])
  data['Availability 60'].append(i['availability']['availability_60'])
  data['Availability 90'].append(i['availability']['availability_90'])
  data['Availability 365'].append(i['availability']['availability_365'])

  # Rating  'review_scores': {'review_scores_rating': 100}}
  data['Rating'].append(i['review_scores']['review_scores_rating'] if 'review_scores_rating' in i['review_scores'] else 0 )
     

### Converting data into structured form

In [14]:
pd.set_option('display.max_columns',None)

df = pd.DataFrame(data)

### Empty string value replace

In [15]:
df.columns

Index(['Id', 'Name', 'Description', 'Property Type', 'Room Type',
       'Minimum Nights', 'Maximum Nights', 'Number Of Reviews', 'Amenities',
       'Price', 'Host ID', 'Host Name', 'Host Neighbourhood',
       'Host Total Listings Count', 'Longitide', 'Latitude', 'Availability 30',
       'Availability 60', 'Availability 90', 'Availability 365', 'Rating'],
      dtype='object')

In [16]:
df['Description'] = df.Description.apply(lambda x : 'not mentioned' if x =='' else x )

In [17]:
df['Name'] = df.Name.apply(lambda x : 'not mentioned' if x =='' else x )

In [18]:
df['Host Neighbourhood'] = df['Host Neighbourhood'].replace("",method='ffill')

In [19]:
df.isnull().sum()

Id                           0
Name                         0
Description                  0
Property Type                0
Room Type                    0
Minimum Nights               0
Maximum Nights               0
Number Of Reviews            0
Amenities                    0
Price                        0
Host ID                      0
Host Name                    0
Host Neighbourhood           0
Host Total Listings Count    0
Longitide                    0
Latitude                     0
Availability 30              0
Availability 60              0
Availability 90              0
Availability 365             0
Rating                       0
dtype: int64

In [20]:
df.columns = ['id', 'name', 'description', 'property_type', 'room_type',
       'minimum_nights', 'maximum_nights', 'number_of_Reviews', 'amenities',
       'price', 'host_iD', 'host_name', 'host_neighbourhood',
       'host_total_listings_count', 'longitide', 'latitude', 'availability_30',
       'availability_60', 'availability_90', 'availability_365', 'rating']

In [21]:
df['minimum_nights'] = df.minimum_nights.apply(lambda x : int(x))

df['maximum_nights'] = df.maximum_nights.apply(lambda x : int(x))

df['number_of_Reviews'] = df.number_of_Reviews.apply(lambda x : int(x))

df['price'] = df.price.astype(str).astype(float)

df.host_total_listings_count = df.host_total_listings_count.apply(lambda x : int(x))

df.longitide	 = df.longitide.apply(lambda x : float(x))

df.latitude = df.latitude.apply(lambda x : float(x))

df.availability_30 = df.availability_30.apply(lambda x : int(x))

df.availability_60 = df.availability_60.apply(lambda x : int(x))

df.availability_90 =  df.availability_90.apply(lambda x : int(x))

df.availability_365 = df.availability_365.apply(lambda x : int(x))

df.rating = df.rating.apply(lambda x : int(x))

df.loc[df.query('host_neighbourhood == "" ').index,'host_neighbourhood']  = 'Copacabana'


In [22]:
df.dtypes

id                            object
name                          object
description                   object
property_type                 object
room_type                     object
minimum_nights                 int64
maximum_nights                 int64
number_of_Reviews              int64
amenities                     object
price                        float64
host_iD                       object
host_name                     object
host_neighbourhood            object
host_total_listings_count      int64
longitide                    float64
latitude                     float64
availability_30                int64
availability_60                int64
availability_90                int64
availability_365               int64
rating                         int64
dtype: object

In [23]:
df

,id,name,description,property_type,room_type,minimum_nights,maximum_nights,number_of_Reviews,amenities,price,host_iD,host_name,host_neighbourhood,host_total_listings_count,longitide,latitude,availability_30,availability_60,availability_90,availability_365,rating
0,1003530,New York City - Upper West Side Apt,"Murphy bed, optional second bedroom available....",Apartment,Private room,12,360,70,"[Internet, Wifi, Air conditioning, Kitchen, El...",135.0,454250,Greta,Copacabana,3,-73.965230,40.799620,0,0,0,93,94
1,10133554,Double and triple rooms Blue mosque,We are on the central city Blue mosque 5 minu...,Bed and breakfast,Private room,1,1125,29,"[Internet, Wifi, Air conditioning, Free parkin...",121.0,52004703,Mehmet Emin,Copacabana,2,28.980090,41.006200,30,60,90,365,92
2,10059872,"Soho Cozy, Spacious and Convenient","Clean, fully furnish, Spacious 1 bedroom flat ...",Apartment,Entire home/apt,4,20,3,"[Air conditioning, Kitchen, Smoking allowed, D...",699.0,51624384,Giovanni,Soho,1,114.150270,22.281580,0,0,0,0,100
3,10084023,City center private room with bed,House is located 5mins walk from Sham Shui Po ...,Guesthouse,Private room,1,500,81,"[TV, Wifi, Air conditioning, Kitchen, Elevator...",181.0,51744313,Yi,Shek Kip Mei,2,114.166900,22.331400,14,24,40,220,92
4,10091713,Surry Hills Studio - Your Perfect Base in Sydney,"This spacious, light filled studio has everyth...",Apartment,Entire home/apt,10,21,64,"[TV, Wifi, Kitchen, Elevator, Heating, Washer,...",181.0,13764143,Ben,Surry Hills,1,151.215540,-33.880290,0,0,0,0,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5550,9985696,Kadıköy-Altıyol,Kadıköy merkezde 2+1 dairem vardır.Çok kalabal...,House,Entire home/apt,1,1125,0,"[TV, Cable TV, Internet, Wifi, Kitchen, Buzzer...",501.0,51303748,Eren,Rio Comprido,2,29.030790,40.985630,30,60,90,365,0
5551,9993190,Just Steps from the Ocean at Premier Platinum ...,Relax Hawaiian Style in Gated Community on Oce...,Townhouse,Entire home/apt,3,365,15,"[TV, Wifi, Air conditioning, Pool, Kitchen, Fr...",220.0,51013381,Coconut Condos,Rio Comprido,58,-156.667520,20.858940,0,8,31,258,92
5552,9887054,Quarry Bay fully furnished studio C,Newly renovated studio flat with full furniture.,Apartment,Private room,3,1125,4,"[TV, Wifi, Air conditioning, Kitchen, Elevator...",400.0,31386255,Cat,Causeway Bay,7,114.207220,22.291570,18,48,78,353,85
5553,9907907,Large double room in Bondi,A bright large double room is available in a b...,Apartment,Private room,3,1125,0,"[TV, Internet, Wifi, Kitchen, Washer, Dryer, S...",60.0,47615848,Lucy,Causeway Bay,1,151.265420,-33.892980,0,0,0,0,0


In [24]:
df.isnull().sum()

id                           0
name                         0
description                  0
property_type                0
room_type                    0
minimum_nights               0
maximum_nights               0
number_of_Reviews            0
amenities                    0
price                        0
host_iD                      0
host_name                    0
host_neighbourhood           0
host_total_listings_count    0
longitide                    0
latitude                     0
availability_30              0
availability_60              0
availability_90              0
availability_365             0
rating                       0
dtype: int64

### Save airbnb data to csv file

In [25]:
df.to_csv('Airbnb.csv',index=False)